# <center>Financial Portfolio Optimization</center>

In [8]:
from ml import data_preprocessing as dp
from ml import forecasting as fc

In [2]:
import importlib
importlib.reload(dp)

<module 'ml.data_preprocessing' from 'd:\\University\\4th Year\\CM4601 Computational Intelligence\\ea-portfolio-optimization\\ml\\data_preprocessing.py'>

In [3]:
selected_symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'JPM']

In [4]:
df = dp.load_data("data/prices.csv")

d:\University\4th Year\CM4601 Computational Intelligence\ea-portfolio-optimization\ml\data_preprocessing.py:14: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_pivoted = df_pivoted.fillna(method='ffill').dropna()


In [5]:
df

symbol,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XLNX,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2016-07-05,43.750000,28.930000,161.190002,94.989998,61.919998,80.360001,39.439999,113.099998,94.730003,55.360001,...,45.570000,93.019997,62.150002,9.24,44.840000,37.500000,83.040002,119.949997,23.440001,47.500000
2016-07-06,44.470001,28.350000,161.449997,95.529999,63.369999,81.190002,40.830002,113.519997,95.180000,55.459999,...,45.480000,94.089996,62.740002,9.32,45.580002,37.509998,84.170004,121.779999,23.990000,48.090000
2016-07-07,44.419998,29.400000,161.130005,95.940002,63.430000,80.989998,41.360001,112.699997,95.139999,55.950001,...,45.709999,92.959999,62.500000,9.38,45.630001,37.520000,84.310004,122.019997,24.410000,47.689999
2016-07-08,45.480000,30.040001,164.850006,96.680000,64.160004,81.919998,41.889999,115.110001,96.790001,57.740002,...,46.970001,93.540001,63.389999,9.54,46.410000,37.740002,85.760004,124.889999,24.740000,48.500000
2016-07-11,45.400002,31.160000,163.690002,96.980003,64.349998,82.180000,42.119999,115.459999,97.139999,58.250000,...,47.220001,93.889999,63.740002,9.50,46.990002,37.959999,86.440005,125.080002,24.930000,48.290001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-23,46.200001,48.480000,170.889999,116.519997,62.340000,78.559998,38.419998,117.480003,105.019997,73.839996,...,60.560001,90.709999,59.240002,8.95,51.009998,38.660000,63.810001,102.320000,43.860001,53.779999
2016-12-27,46.509998,48.610001,171.839996,117.260002,62.500000,78.820000,38.599998,117.550003,104.980003,74.309998,...,61.240002,90.750000,59.070000,8.94,50.860001,38.919998,64.029999,103.139999,43.930000,53.720001
2016-12-28,45.720001,47.669998,170.419998,116.760002,62.270000,78.349998,38.220001,116.610001,103.769997,73.559998,...,60.630001,90.300003,58.150002,8.76,49.939999,38.730000,63.720001,103.059998,43.279999,53.439999


In [6]:
returns = dp.compute_returns(df)
returns

symbol,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XLNX,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2016-07-06,0.016457,-0.020048,0.001613,0.005685,0.023417,0.010329,0.035243,0.003714,0.004750,0.001806,...,-0.001975,0.011503,0.009493,0.008658,0.016503,0.000267,0.013608,0.015256,0.023464,0.012421
2016-07-07,-0.001124,0.037037,-0.001982,0.004292,0.000947,-0.002463,0.012981,-0.007223,-0.000420,0.008835,...,0.005057,-0.012010,-0.003825,0.006438,0.001097,0.000267,0.001663,0.001971,0.017507,-0.008318
2016-07-08,0.023863,0.021769,0.023087,0.007713,0.011509,0.011483,0.012814,0.021384,0.017343,0.031993,...,0.027565,0.006239,0.014240,0.017058,0.017094,0.005864,0.017198,0.023521,0.013519,0.016985
2016-07-11,-0.001759,0.037284,-0.007037,0.003103,0.002961,0.003174,0.005491,0.003041,0.003616,0.008833,...,0.005323,0.003742,0.005521,-0.004193,0.012497,0.005829,0.007929,0.001521,0.007680,-0.004330
2016-07-12,0.024890,0.112323,-0.000855,0.004537,0.017871,0.024337,0.008310,0.009007,0.004118,0.012704,...,0.005506,0.011290,-0.009570,0.017895,0.002766,-0.001844,-0.002429,0.003038,0.027276,0.012011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-23,0.005003,-0.004313,0.002934,0.001978,0.011028,0.009639,0.003395,-0.002632,0.002865,0.005310,...,0.006147,-0.001761,0.001860,-0.006659,-0.002347,0.004156,-0.003747,0.007384,0.002743,0.009763
2016-12-27,0.006710,0.002682,0.005559,0.006351,0.002567,0.003310,0.004685,0.000596,-0.000381,0.006365,...,0.011229,0.000441,-0.002870,-0.001117,-0.002941,0.006725,0.003448,0.008014,0.001596,-0.001116
2016-12-28,-0.016986,-0.019338,-0.008263,-0.004264,-0.003680,-0.005963,-0.009844,-0.007997,-0.011526,-0.010093,...,-0.009961,-0.004959,-0.015575,-0.020134,-0.018089,-0.004882,-0.004841,-0.000776,-0.014796,-0.005212


In [7]:
norm_returns = dp.normalize_returns(returns)
norm_returns

symbol,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XLNX,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2016-07-06,1.205501,-1.139093,0.058210,0.331920,1.589989,0.480051,2.866901,0.286130,0.336671,-0.033969,...,-0.370805,1.118739,0.900050,0.720234,1.317606,-0.005822,0.539968,0.902251,1.154848,0.911975
2016-07-07,-0.115576,1.558530,-0.137697,0.217043,0.051390,-0.115331,1.062826,-0.644091,-0.097004,0.486391,...,0.236462,-1.131578,-0.296176,0.543212,0.019553,-0.005820,0.117972,0.166298,0.782283,-0.748990
2016-07-08,1.761986,0.837013,1.228429,0.499197,0.774590,0.533780,1.049345,1.789084,1.392855,2.200784,...,2.180163,0.614974,1.326395,1.389936,1.367386,0.381532,0.666818,1.360053,0.532841,1.277477
2016-07-11,-0.163256,1.570181,-0.413153,0.118999,0.189321,0.147047,0.455868,0.228892,0.241537,0.486201,...,0.259381,0.375940,0.543321,-0.304375,0.980095,0.379157,0.339339,0.141404,0.167634,-0.429602
2016-07-12,1.839127,5.116260,-0.076297,0.237255,1.210225,1.132047,0.684312,0.736397,0.283619,0.772794,...,0.275233,1.098343,-0.812156,1.456684,0.160220,-0.151896,-0.026621,0.225420,1.393285,0.879112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-23,0.344857,-0.395502,0.130223,0.026196,0.741682,0.447952,0.286059,-0.253559,0.178522,0.225393,...,0.330598,-0.150685,0.214493,-0.501023,-0.270619,0.263343,-0.073174,0.466167,-0.141110,0.699128
2016-12-27,0.473092,-0.064970,0.273254,0.386851,0.162297,0.153365,0.390594,0.020962,-0.093696,0.303527,...,0.769400,0.060040,-0.210344,-0.059160,-0.320626,0.441165,0.181014,0.501066,-0.212880,-0.172173
2016-12-28,-1.307377,-1.105506,-0.480006,-0.488573,-0.265418,-0.278214,-0.786810,-0.709856,-1.028487,-0.914882,...,-1.060431,-0.456740,-1.351466,-1.575390,-1.596954,-0.362129,-0.111836,0.014162,-1.238121,-0.500271


---

In [9]:
expected_returns, expected_volatility = fc.calculate_forecast(norm_returns)

In [10]:
expected_returns

symbol
A       0.171851
AAL    -0.134994
AAP    -0.053349
AAPL    0.097172
ABBV    0.185043
          ...   
YHOO   -0.098913
YUM     0.077771
ZBH     0.153529
ZION   -0.108627
ZTS     0.262869
Name: 2016-12-30 00:00:00, Length: 501, dtype: float64